In [ ]:
!pip install -q pyjwt

In [ ]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import requests, jwt

CATALOG_URL = "http://server:8080/catalog"
MANAGEMENT_URL = "http://server:8080/management"
KEYCLOAK_TOKEN_URL = "http://keycloak:8080/realms/iceberg/protocol/openid-connect/token"

# Bootstraping Lakekeeper
Initially, the Lakekeeper needs to be bootstrapped once.
During bootstrapping the initial `Global Administrator` is set. Bootstrapping can only be performed once. The first user calling the bootstrap endpoint, will become the `Global Administrator`.

## 1. Sign in
First, we need to obtain a token from our Identity Provider. In this example a `Keycloak` is running as a pod beside Lakekeeper. A few users have been pre-created in Keycloak for this example. We are now logging into Keycloak as the technical user (client) `iceberg-machine-client`. If a human user bootstraps the catalog, we recommend to use the UI.

Keycloak can be accessed at http://localhost:30080 in this example. Use `admin` as username and password. Then select the `iceberg` realm on the top left corner.

In [ ]:
# Login to Keycloak
CLIENT_ID = "iceberg-machine-client"
CLIENT_SECRET = "M2HyiuJRJupuOEaCVni2wLDXAf2GZeCi"

response = requests.post(
    url=KEYCLOAK_TOKEN_URL,
    data={
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        # For bootstraping the token must contain something we can use as a user name.
        # Especially for human husers this can be achieved with the "profile" OIDC scope.
        # Scopes might be different depending on your IdP configuration.
        "scope": "profile"
    },
    headers={"Content-type": "application/x-www-form-urlencoded"},
)
response.raise_for_status()
access_token = response.json()['access_token']

# Lets inspect the token we got to see that our application name is available:
jwt.decode(access_token, options={"verify_signature": False})

Now that we have the access token, we can query the server info Endpoint. 
On first launch it will show bootstrapped
The full API documentation is available as part of the Repository and hosted by Lakekeeper: http://localhost:8080/swagger-ui/#/

In [ ]:
response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/info",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
response.json()
# On first launch it shows "bootstrapped": False

## 2. Bootstrap

In [ ]:
response = requests.post(
    url=f"{MANAGEMENT_URL}/v1/bootstrap",
    headers={
        "Authorization": f"Bearer {access_token}"
    },
    json={
        "accept-terms-of-use": True,
        # Optionally, we can override the name / type of the user:
        # "user-email": "user@example.com",
        # "user-name": "Roald Amundsen",
        # "user-type": "human"
    },
)
response.raise_for_status()

## 3. Validate Bootstrap

In [ ]:
# The server is now bootstrapped:
response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/info",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
response.json()

In [ ]:
# An initial user was created
response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/user",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
response.json()

In [ ]:
# This user is the global admin, which has all access rights to the server:
response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/permissions/server/access",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
response.json()

In [ ]:
response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/permissions/server/assignments",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
print(response.json())
user_id = response.json()['assignments'][0]['user']

In [ ]:
# Lets see who this user is:
response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/user/{user_id}",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
print(response.json())